In [13]:
import h5py
from prepare_individuals import prepare, germanBats
import pickle
import numpy as np

classes = germanBats

In [2]:
prepared_hf = h5py.File('prepared.h5', 'r')

patch_len = 44                               # 88 bei 44100, 44 bei 22050 = 250ms ~ 25ms

In [3]:
X_train, Y_train, X_test, Y_test, X_val, Y_val = prepare("../datasets/prepared.h5", classes, patch_len=patch_len)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  4.76it/s]


In [4]:
print("Total calls:", len(X_train) + len(X_test) + len(X_val))
print(X_train.shape, Y_train.shape)

Total calls: 54280
(31872, 44, 257) (31872,)


In [29]:
# inverse distribution sampling
grouped = [[] for x in range(len(classes))]
for x, y in zip(X_train, Y_train):
    grouped[y].append((x, y))
sorted_list = list(sorted(grouped, key=len)) # class with least elements first
sampling = []
size = 1.0
for group in sorted_list:
    take = int(len(group) * size)
    print(len(group), "take", take)
    samples = np.asarray(group)[range(take)]
    sampling.extend(samples)
    size = size / 2
    
print(len(sampling))

72 take 72
278 take 139
439 take 109
476 take 59
566 take 35
1102 take 34
1204 take 18
1227 take 9
1300 take 5
1353 take 2
1368 take 1
1381 take 0
1415 take 0
1669 take 0
2396 take 0
2587 take 0
3053 take 0
4213 take 0
5773 take 0
483


In [20]:
from ipyannotate import annotate
from ipyannotate.buttons import (
    ValueButton as Button,
    NextButton as Next,
    BackButton as Back
)

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display

In [32]:
def display_item(a):
    patch, label = a
    plt.figure(figsize = (2.5, 5))
    spec = np.rot90(patch)
    plt.imshow(spec, interpolation='nearest', aspect='auto', cmap='inferno')
    plt.title(list(classes)[label])
    plt.colorbar()
    plt.savefig('out.png')
    plt.close()
    image = Image.open('out.png')
    display(image)

buttons = [
    Button('Call', shortcut='1'),
    Button('No Call', shortcut='2'),
    Back(),
    Next()
]    
annotation = annotate(sampling + list(zip(X_train[:2000], Y_train[:2000])), buttons=buttons, display=display_item)

# load annotations
'''with open('call_nocall.annotation', 'rb') as file:
    values = pickle.load(file)
    for i, val in enumerate(values):
        annotation.tasks[len(sampling) + i].value = "Call" if val == 1 else "No Call"
with open('call_nocall.tasks', 'wb') as file:
    pickle.dump(annotation.tasks, file)'''
with open('call_nocall.tasks', 'rb') as file:
    annotation.tasks = pickle.load(file)
annotation

Annotation(canvas=OutputCanvas(), progress=Progress(atoms=[<ipyannotate.progress.Atom object at 0x7efe1e460880…

In [33]:
call = 0
no_call = 0
for task in annotation.tasks:
    if task.value == "Call":
        call += 1
    elif task.value == "No Call":
        no_call += 1

print(call, no_call, call + no_call)

1431 1052 2483


In [34]:
with open('call_nocall.annotation', 'wb') as file:
    values = []
    for task in annotation.tasks:
        values.append(1 if task.value == "Call" else 0)
    pickle.dump(np.asarray(values), file)

In [35]:
with open('call_nocall.annotation', 'rb') as file:
    values = pickle.load(file)
    call = values.sum()
    no_call = values.shape[0] - call
    print(call, no_call)

1431 1052
